In [10]:
import os
import time
import re
from PIL import Image
import requests
import urllib.request
from bs4 import BeautifulSoup as bs
from selenium import webdriver

### 29cm URL 만들기~

In [11]:
top = {"short_sleeve":"01", "long_sleeve":"02", "pique_tshirts":"03", "sleeveless":"04", "sweat_shirts":"05", "hoodie":"06", "zipup":"07"}
shirts = {"short_shirts":"01", "long_shirts":"02", "dress_shirts":"03"}
pants = {"slacks":"01", "chino":"02", "short":"03", "cargo":"04", "denim":"05", "training":"06"}
knit = {"crew":"01", "cardigan":"02", "turtle":"03", "vest":"04"}
shoes = {"rain":"12", "sneakers":"01", "running":"09", "derby":"13", "boots":"05", "loafers":"03", "flipflop":"10", "sandle":"02"}
outer = {"light_padding":"11", "fleece":"10", "blazer":"01", "trench_coat":"09", "trucker":"02", "coat":"06", "padding":"07", "robe":"04", "vest":"05"}
bags = {"shoulder":"02", "tote":"05", "cross":"11", "backpack":"04"}
medium_category = {"top":("2551181", top), "shirts":("2551021", shirts), "knit":("2551041", knit), "pants":("2551031", pants), "shoes":("2551061", shoes), "outer":("2551051", outer), "bags":("2551071", bags)}
# medium_category["top"][0] + medium_category["top"][1]["short_sleeve"]

# url = f"https://www.29cm.co.kr/shop/category/list?category_large_code=255100100&category_medium_code={medium_category}00&category_small_code={medium_category}{small_category}&sort=latest&page={page}&brand=&min_price=&max_price=&count=50"

In [12]:
# path = "/root/chrome_driver/chromedriver"
path_window = r"/Users/seungsu/Desktop/YBIGTA/chromedriver.exe"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("window-size=1920x1080")
# chrome_options.add_argument("headless")

driver = webdriver.Chrome(path_window, options=chrome_options)
driver.implicitly_wait(3)

#### 아이템 태그 리스트 얻기

In [13]:
driver.get("https://www.29cm.co.kr/shop/category/list?category_large_code=255100100&category_medium_code=255118100&category_small_code=255118101&sort=latest&page=1&brand=&min_price=&max_price=&count=50")
ul = driver.find_element_by_css_selector("ul.prd_section")
all_items = ul.find_elements_by_css_selector("a.prd_b_area")

#### 아이템 태그 분석

In [14]:
items = []
for a in all_items:
    item_url = a.get_attribute("href")     #세부 아이템 url
    item_img = a.find_element_by_css_selector("div.imgbx").find_element_by_css_selector("img.b-lazy").get_attribute("src")
    item_brand = a.find_element_by_css_selector("div.brand").text
    item_name = a.find_element_by_css_selector("div.name").text
    img = item_img
    items.append({"brand":item_brand, "name":item_name, "href":item_url, "img":img})

In [15]:
for i in items:
    print(i)

{'brand': '더니트컴퍼니', 'name': '[스페셜 오더]_수피마 쿨맥스 티셔츠 16컬러', 'href': 'https://www.29cm.co.kr/product/663865', 'img': 'https://img.29cm.co.kr/next-product/2020/04/23/4a2540877c2a43a98ebd21edebdf4225_20200423183038.jpg?width=500'}
{'brand': '플루크', 'name': '플루크 new 피그먼트 오버핏 반팔티셔츠 FST701 / 7color M', 'href': 'https://www.29cm.co.kr/product/611019', 'img': 'https://img.29cm.co.kr/next-product/2020/03/05/5a6de5a710f6461ebb52e31c6f79a5e3_20200305150104.jpg?width=500'}
{'brand': '플루크', 'name': '플루크 new 피그먼트 오버핏 반팔티셔츠 FST701 / 7color', 'href': 'https://www.29cm.co.kr/product/611034', 'img': 'https://img.29cm.co.kr/next-product/2020/03/05/a16e2a4490d947a6b98e15b8a621a15d_20200305150404.jpg?width=500'}
{'brand': '네이더스', 'name': '505C-1 STANDARD SS T-SHIRT', 'href': 'https://www.29cm.co.kr/product/601122', 'img': 'https://img.29cm.co.kr/next-product/2020/02/28/3785e59337294e2fa73b3f6f8efc33b5_20200228181828.jpg?width=500'}
{'brand': '네이더스', 'name': '505C-2 STANDARD SS T-SHIRT', 'href': 'https://www.29

#### 세부 데이터 URL

In [23]:
item_detail_url = items[4]["href"]
item_detail_url

'https://www.29cm.co.kr/product/601123'

#### 세부 데이터 크롤링

In [31]:
# 데이터 저장 dict
detail_item_info = {}

# 접속
driver.get(item_detail_url)

# 기본정보
detail_item = driver.find_element_by_css_selector("div.detail_item")
detail_item_info["shop"] = "29cm"
detail_item_info["brand"] = detail_item.find_element_by_css_selector("h1.kor").text
# detail_item_info["brand"] = detail_item.find_element_by_css_selector("h1.eng").text
detail_item_info["product"] = detail_item.find_element_by_css_selector("div.name").text
detail_item_info["product_id"] = driver.find_element_by_css_selector("em.dsc").text

# 이미지url
detail_item_img = detail_item.find_element_by_css_selector("img.b-lazy.b-loaded").get_attribute("src")
detail_item_info["img"] = detail_item_img

# 상세정보
infos = driver.find_element_by_css_selector("div.item_info").find_elements_by_css_selector("tr")
for row in infos:
    key = row.find_element_by_css_selector("th").find_element_by_css_selector("div").text
    value = row.find_element_by_css_selector("td").find_element_by_css_selector("div").text
    detail_item_info[key] = value

In [32]:
detail_item_info

{'shop': '29cm',
 'brand': '네이더스',
 'product': '505C-2 STANDARD SS T-SHIRT',
 'product_id': '601123',
 'img': 'https://img.29cm.co.kr/next-product/2020/02/27/9a4f89337cbf4724a3ab82b2486bcc79_20200227154352.jpg?width=700',
 '제품 소재': 'COTTON 100%',
 '색상': 'NAVY',
 '치수': '0,3,4,5',
 '제조자': 'N/BENEDEF INNOVATIONS',
 '제조국': 'KOREA',
 '세탁방법 및 취급시 주의사항': '케어라벨 참조',
 '제조연월': '-',
 '품질보증기준': '* 교환 및 반품은 제품수령일 기준으로 7일 이내에 보내주셔야 가능합니다. * 종이택(가격택)을 제거시 재판매가 불가능하여 교환 및 반품이 불가능합니다.',
 'A/S 책임자와 전화번호': '29CM 고객행복센터 1644-0560'}

In [12]:
def getBSobj(category_name, page="1"):
    url = f"https://www.29cm.co.kr/shop/category/list?category_large_code=255100100&category_medium_code={category_name[:7]}00&category_small_code={category_name}&sort=latest&page={page}&brand=&min_price=&max_price=&count=50"
    print(url)
    session = requests.Session()
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    html = session.get(url, headers=headers).content
    bsObj = bs(html, "html.parser")
    return bsObj

In [15]:
cate = medium_category["top"][0] + medium_category["top"][1]["short_sleeve"]
cate
bsobj = getBSobj(cate)
bsobj.find("ui-root")

https://www.29cm.co.kr/shop/category/list?category_large_code=255100100&category_medium_code=255118100&category_small_code=255118101&sort=latest&page=1&brand=&min_price=&max_price=&count=50


<ui-root></ui-root>

### 저장용 디렉토리 생성

In [5]:
def check_directory(category_name, folder_path=None):
    if folder_path == None:
        folder_path = "C:/Users/seungsu/Desktop/ICT-project/crawling"
    try:
        os.mkdir(f"{folder_path}/{category_name}/")
        print("Directory is created!")
    except:
        print("Directory exist!")

### 단일페이지 크롤러

In [6]:
def crawl_page(bsObj, category_name):
    body = bsObj.find("ul", {"class":"prd_section"})
    all_items = body.find_all("a", {"class":"prd_b_area"})
    items = []
    for a in all_items:
        item_url = a["href"]     #세부 아이템 url
        item_img = a.find("div", {"class":"imgbx"}).find("img", {"class":"b-lazy"})["src"]
        item_brand = a.find("div", {"class":"brand"})
        item_name = a.find("div", {"class":"name"})
        img = "http:" + item_img
        items.append([item_brand, item_name, item_url, img])
#         image = urllib.request.urlretrieve(img, f"./{category_name}/{id}.png")
#         #image resize
#         image = Image.open(f"./{category_name}/{id}.png")
#         resize_img = image.resize((125,125))
#         resize_img.save(f"./{category_name}/{id}.png")
        print(item_url)
        print(item_brand, item_name, item_url, img)
    return items
    
# crawl_page(getBSobj(page="1",category="001004"), category_name="상의")

In [9]:
crawl_page(bsobj, cate)

AttributeError: 'NoneType' object has no attribute 'find_all'